#### Triple A to MLB Translation

In [36]:
import pandas as pd
import numpy as np
import os

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/') 

In [37]:
minor = pd.read_csv('./minor_leauge_preds/data/minor_clean.csv')
major = pd.read_csv('./major_leauge_preds/data/cleaned/data_02_24.csv')

In [38]:
minor_aaa = minor[minor['Level'] == 'AAA']
minor_aaa = minor_aaa.sort_values(['Name', 'Season'])
minor_aaa['Season_diff'] = minor_aaa.groupby('Name')['Season'].diff()
minor_aaa['group'] = (minor_aaa['Season_diff'] != 1).cumsum()
minor_aaa = minor_aaa.groupby('Name').apply(lambda x: x[x['group'] == x['group'].iloc[0]]).reset_index(drop=True)
minor_aaa = minor_aaa.drop(['Season_diff', 'group'], axis=1)

C:\Users\dalto\AppData\Local\Temp\ipykernel_10176\3590174584.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  minor_aaa = minor_aaa.groupby('Name').apply(lambda x: x[x['group'] == x['group'].iloc[0]]).reset_index(drop=True)


In [39]:
from tools.zscore_pa import normalize
minor_aaa = normalize(minor_aaa)

In [40]:
from tools.three_yr_avg import three_year_avg
minor_aaa = three_year_avg(minor_aaa)

MLB Data Frame

In [32]:
major = major[major['PA'] > 100]
major = major.sort_values(by=['Name'])
major = major.drop_duplicates(subset='Name', keep='first')
major = major.drop(columns=['Unnamed: 0'])
major['Team'] = 'MLB'
major = major.rename(columns={'Team':'Level'})
major = major[['Name', 'Level', 'wRC+']]


Merge

In [35]:
outcome = minor_aaa.merge(major[['Name','wRC+']], on=['Name'], how='outer')